# Part 7 - Federated Learning with FederatedDataset

Here we introduce a new tool for using federated datasets. We have created a `FederatedDataset` class which is intended to be used like the PyTorch Dataset class, and is given to a federated data loader `FederatedDataLoader` which will iterate on it in a federated fashion.


Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

We use the sandbox that we discovered last lesson

In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


Then search for a dataset

In [2]:
boston_data = grid.search("#boston", "#data", verbose=False, return_counter=False)
boston_target = grid.search("#boston", "#target", verbose=False, return_counter=False)

We load a model and an optimizer

In [3]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

Here we cast the data fetched in a `FederatedDataset`. See the workers which hold part of the data.

In [4]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


We put it in a `FederatedDataLoader` and specify options

In [5]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

And finally we iterate over epochs. You can see how similar this is compared to pure and local PyTorch training!

In [6]:
epochs = 50
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)


Train Epoch: 1 [0/16 (0%)]	Batch loss: 3156.205322
Train Epoch: 1 [8/16 (50%)]	Batch loss: 59.617584
Train Epoch: 1 [16/16 (100%)]	Batch loss: 879.110596
Total loss 25813.378814697266
Train Epoch: 2 [0/16 (0%)]	Batch loss: 1049.295288
Train Epoch: 2 [8/16 (50%)]	Batch loss: 142.419449
Train Epoch: 2 [16/16 (100%)]	Batch loss: 385.578247
Total loss 12938.65673828125
Train Epoch: 3 [0/16 (0%)]	Batch loss: 1212.881592
Train Epoch: 3 [8/16 (50%)]	Batch loss: 61.300438
Train Epoch: 3 [16/16 (100%)]	Batch loss: 174.486832
Total loss 9758.002326965332
Train Epoch: 4 [0/16 (0%)]	Batch loss: 1113.664429
Train Epoch: 4 [8/16 (50%)]	Batch loss: 72.734505
Train Epoch: 4 [16/16 (100%)]	Batch loss: 156.222260
Total loss 8611.730628967285
Train Epoch: 5 [0/16 (0%)]	Batch loss: 704.376953
Train Epoch: 5 [8/16 (50%)]	Batch loss: 108.787155
Train Epoch: 5 [16/16 (100%)]	Batch loss: 59.056713
Total loss 5583.505939483643
Train Epoch: 6 [0/16 (0%)]	Batch loss: 588.390381
Train Epoch: 6 [8/16 (50%)]	Batch 

Train Epoch: 46 [8/16 (50%)]	Batch loss: 41.801105
Train Epoch: 46 [16/16 (100%)]	Batch loss: 15.974016
Total loss 815.01096534729
Train Epoch: 47 [0/16 (0%)]	Batch loss: 30.604919
Train Epoch: 47 [8/16 (50%)]	Batch loss: 41.727428
Train Epoch: 47 [16/16 (100%)]	Batch loss: 15.885193
Total loss 813.4740800857544
Train Epoch: 48 [0/16 (0%)]	Batch loss: 30.547512
Train Epoch: 48 [8/16 (50%)]	Batch loss: 41.667236
Train Epoch: 48 [16/16 (100%)]	Batch loss: 15.789288
Total loss 811.7557349205017
Train Epoch: 49 [0/16 (0%)]	Batch loss: 30.476841
Train Epoch: 49 [8/16 (50%)]	Batch loss: 41.609627
Train Epoch: 49 [16/16 (100%)]	Batch loss: 15.689669
Total loss 809.9440565109253
Train Epoch: 50 [0/16 (0%)]	Batch loss: 30.398361
Train Epoch: 50 [8/16 (50%)]	Batch loss: 41.546104
Train Epoch: 50 [16/16 (100%)]	Batch loss: 15.590181
Total loss 808.1320648193359


# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)